# 0. GPU?

In [9]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "6"

# 1. Document 확인


In [71]:
import requests
from bs4 import BeautifulSoup

In [72]:
# section_num = '100'

# base_url = 'https://news.naver.com/section/'

# # 섹션 번호를 URL에 추가
# url = base_url + section_num

# # URL에 요청 보내기
# response = requests.get(url)

# # 페이지의 HTML 소스 가져오기
# html = response.text

# # BeautifulSoup을 사용해 HTML 파싱
# soup = BeautifulSoup(html, 'html.parser')

# # 파싱된 HTML 문서 전체를 출력
# print(soup.prettify())

In [73]:
# import requests
# from bs4 import BeautifulSoup

# # 네이버 뉴스에서 섹션별로 최신 기사를 크롤링하는 함수
# def crawl_latest_news(section_num):
#     # 기본 URL 설정
#     base_url = 'https://news.naver.com/section/'
    
#     # 섹션 번호를 URL에 추가
#     url = base_url + section_num
#     print(f"크롤링할 URL: {url}")

#     # URL에 요청 보내기
#     response = requests.get(url)
    
#     # 요청이 성공적인지 확인
#     if response.status_code == 200:
#         # 페이지의 HTML 소스 가져오기
#         html = response.text
        
#         # BeautifulSoup을 사용해 HTML 파싱
#         soup = BeautifulSoup(html, 'html.parser')
        
#         # 최신 기사 목록을 포함하는 div를 찾기
#         latest_articles = soup.find_all('div', class_='section_latest_article _CONTENT_LIST _PERSIST_META')
        
#         # 추출된 최신 기사 목록을 출력 (현재는 각 기사의 링크 및 제목을 출력)
#         for article in latest_articles:
#             title = article.find('a').get_text().strip()
#             link = article.find('a')['href']
#             print(f"기사 제목: {title}, 링크: {link}")
        
#     else:
#         print(f"요청 실패. 상태 코드: {response.status_code}")

In [74]:
# # 정치 뉴스 섹션(섹션 번호: '100') 크롤링 예시
# crawl_latest_news('100')

In [75]:
# import requests
# from bs4 import BeautifulSoup

# # 네이버 뉴스 섹션별로 여러 페이지를 크롤링하는 함수
# def crawl_section_news(section_num, max_pages=5):
#     base_url = 'https://news.naver.com/main/main.nhn?sid1='
    
#     # 섹션 번호를 URL에 추가
#     for page in range(1, max_pages + 1):
#         url = f'{base_url}{section_num}&page={page}'
#         print(f"크롤링할 페이지 URL: {url}")

#         # URL에 요청 보내기
#         response = requests.get(url)

#         # 요청이 성공적인지 확인
#         if response.status_code == 200:
#             # 페이지의 HTML 소스 가져오기
#             html = response.text

#             # BeautifulSoup으로 HTML 파싱
#             soup = BeautifulSoup(html, 'html.parser')

#             # 최신 기사 목록을 포함하는 div를 찾기 (div class='section_latest_article _CONTENT_LIST _PERSIST_META')
#             latest_articles = soup.find_all('div', class_='section_latest_article _CONTENT_LIST _PERSIST_META')

#             # 추출된 최신 기사 목록을 출력 (현재는 각 기사의 링크 및 제목을 출력)
#             for article in latest_articles:
#                 title = article.find('strong').get_text()
#                 link = article.find('a')['href']
#                 print(f"기사 제목: {title}, 링크: {link}")
#         else:
#             print(f"요청 실패. 상태 코드: {response.status_code}")

# # 정치 뉴스 섹션(섹션 번호: '100')을 5페이지까지 크롤링 예시
# crawl_section_news('100', max_pages=5)


# 2. 크롤링 수행

In [111]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re

def get_article_time_from_text(time_text):
    """시간 텍스트를 기반으로 실제 시간을 계산하여 반환"""
    if '분전' in time_text:
        minutes_ago = int(re.search(r'\d+', time_text).group())
        return datetime.now() - timedelta(minutes=minutes_ago)
    elif '시간전' in time_text:
        hours_ago = int(re.search(r'\d+', time_text).group())
        return datetime.now() - timedelta(hours=hours_ago)
    elif '일전' in time_text:
        days_ago = int(re.search(r'\d+', time_text).group())
        return datetime.now() - timedelta(days=days_ago)
    else:
        # 다른 형식의 시간 텍스트 처리 필요시 추가
        return None

def crawl_news_articles(section_num, target_time, max_pages=10):
    base_url = 'https://news.naver.com/section/template/SECTION_ARTICLE_LIST'
    
    articles = []
    next_cursor = None  # 초기 next 값은 None으로 시작
    page = 1

    while True:
        # AJAX 요청에 필요한 파라미터 설정
        params = {
            'sid': section_num,
            'sid2': '',
            'cluid': '',
            'pageNo': page,
            'date': '',
            'next': next_cursor,
            '_': int(datetime.now().timestamp() * 1000)  # 현재 타임스탬프
        }

        # AJAX 요청 보내기
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            # JSON 응답을 디코딩
            data = json.loads(response.text)
            
            # HTML 콘텐츠가 포함된 부분을 추출
            html_content = data.get('renderedComponent')['SECTION_ARTICLE_LIST']
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # 각 기사에 대한 정보를 추출
            article_blocks = soup.find_all('li')  # 필요한 li 태그로 크롤링 (html 구조에 따라 조정 필요)
            for block in article_blocks:
                title = block.find('strong').get_text().strip()
                link = block.find('a')['href']

                # "n분전", "n시간전" 텍스트를 추출하여 발행 시간 계산
                time_text = block.find('b').get_text()
                
                article_time = get_article_time_from_text(time_text)

                if article_time and article_time < target_time:
                    print("목표 시간에 도달했습니다. 크롤링을 종료합니다.")
                    return articles

                articles.append((title, link, article_time))
                print(f"기사 제목: {title}, 링크: {link}, 발행 시간: {article_time}")

            # "더 보기" 버튼이 있는지 확인
            more_button = soup.find('a', class_='section_more_inner _CONTENT_LIST_LOAD_MORE_BUTTON')


            if more_button:
                # 다음 페이지로 넘어가기 위한 next_cursor 값 갱신
                next_cursor_element = soup.find('div', class_='section_latest_article _CONTENT_LIST _PERSIST_META')
                if next_cursor_element:
                    next_cursor = next_cursor_element.get('data-cursor')  # 다음 AJAX 요청에 사용할 next 값 갱신
                    print(f"Next Cursor: {next_cursor}")
                else:
                    print("더 이상 불러올 데이터가 없습니다.")
                    break
            else:
                print("더 보기 버튼이 존재하지 않음. 크롤링을 종료합니다.")
                break

            page += 1

            if page > max_pages:
                print(f"최대 페이지({max_pages})에 도달했습니다. 크롤링을 종료합니다.")
                break
        else:
            print(f"요청 실패. 상태 코드: {response.status_code}")
            break

    return articles


# 본문 크롤링 수행 코드
def get_article_content(article_url):
    """기사의 URL에서 본문 내용을 가져오는 함수"""
    try:
        response = requests.get(article_url)

        if response.status_code == 200:
            # HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # article 태그 내의 본문 내용 추출
            article_tag = soup.find('article')
            
            if article_tag:
                # 본문 텍스트만 추출 (HTML 태그 제외)
                content = article_tag.get_text(separator="\n").strip()
                return content
            else:
                print(f"본문 내용을 찾을 수 없습니다: {article_url}")
                return None
        else:
            print(f"요청 실패: {response.status_code}")
            return None
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [112]:
# 목표 시간 설정 (예: 1시간 전)
target_time = datetime.now() - timedelta(hours=1)

# 정치 섹션(섹션 번호: '100')에 대해 크롤링 수행
articles = crawl_news_articles('100', target_time, max_pages=20)

기사 제목: 김경율 "이스타항공 탈세 제보, 文 정부 국세청 뭉갰다", 링크: https://n.news.naver.com/mnews/article/448/0000478163, 발행 시간: 2024-09-19 13:33:06.935539
기사 제목: [단독] 김건희 여사 공천 개입 의혹 'M'씨, 뉴스토마토 기자 고소, 링크: https://n.news.naver.com/mnews/article/629/0000322415, 발행 시간: 2024-09-19 13:33:06.935779
기사 제목: 김민석 "금투세 3년 유예해 증시 부양하자"…유예론 가세, 링크: https://n.news.naver.com/mnews/article/277/0005473739, 발행 시간: 2024-09-19 13:31:06.935876
기사 제목: 광주 찾은 문재인 전 대통령… 9·19 평양 공동선언 6주년 기념식 참석 [뉴시스Pic], 링크: https://n.news.naver.com/mnews/article/003/0012791969, 발행 시간: 2024-09-19 13:29:06.935964
기사 제목: [류병수의 강펀치] 김경율 "타이이스타젯 71억원 중 사라진 50억원에 대한 특검 필요", 링크: https://n.news.naver.com/mnews/article/448/0000478162, 발행 시간: 2024-09-19 13:22:06.936053
기사 제목: 천하람 "곽튜브 편집 요구 과해…사소한 일로 나락 일상화", 링크: https://n.news.naver.com/mnews/article/088/0000905384, 발행 시간: 2024-09-19 13:22:06.936142
기사 제목: [무등의 아침] 폭염 속 타들어 가는 農心…정치 9단, 박지원이 느낀 추석 민심은?, 링크: https://n.news.naver.com/mnews/article/056/0011802960, 발행 시간: 2024-09-19 13:20:06.936229
기사 제목: 이언주

In [113]:
# 예시: 기사 링크 리스트에서 본문을 수집하는 코드
articles = crawl_news_articles('100', target_time, max_pages=20)

for title, link, _ in articles:
    print(f"기사 제목: {title}")
    content = get_article_content(link)
    if content:
        print(f"기사 본문:\n{content}")
        print("="*40)

기사 제목: 김경율 "이스타항공 탈세 제보, 文 정부 국세청 뭉갰다", 링크: https://n.news.naver.com/mnews/article/448/0000478163, 발행 시간: 2024-09-19 13:33:08.142014
기사 제목: [단독] 김건희 여사 공천 개입 의혹 'M'씨, 뉴스토마토 기자 고소, 링크: https://n.news.naver.com/mnews/article/629/0000322415, 발행 시간: 2024-09-19 13:33:08.142441
기사 제목: 김민석 "금투세 3년 유예해 증시 부양하자"…유예론 가세, 링크: https://n.news.naver.com/mnews/article/277/0005473739, 발행 시간: 2024-09-19 13:31:08.142541
기사 제목: 광주 찾은 문재인 전 대통령… 9·19 평양 공동선언 6주년 기념식 참석 [뉴시스Pic], 링크: https://n.news.naver.com/mnews/article/003/0012791969, 발행 시간: 2024-09-19 13:29:08.142638
기사 제목: [류병수의 강펀치] 김경율 "타이이스타젯 71억원 중 사라진 50억원에 대한 특검 필요", 링크: https://n.news.naver.com/mnews/article/448/0000478162, 발행 시간: 2024-09-19 13:22:08.142727
기사 제목: 천하람 "곽튜브 편집 요구 과해…사소한 일로 나락 일상화", 링크: https://n.news.naver.com/mnews/article/088/0000905384, 발행 시간: 2024-09-19 13:22:08.142821
기사 제목: [무등의 아침] 폭염 속 타들어 가는 農心…정치 9단, 박지원이 느낀 추석 민심은?, 링크: https://n.news.naver.com/mnews/article/056/0011802960, 발행 시간: 2024-09-19 13:20:08.142910
기사 제목: 이언주